In [1]:
import cv2
import os
import keras
import time
import imutils
import itertools
import numpy as np
import pandas as pd
import seaborn as sn
from collections import Counter
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [2]:
# Import custom metrics used in training model
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
      f1_score,
]

In [3]:
# Load model
model_dir = 'same_model.h5'
model = keras.models.load_model(model_dir, custom_objects={"f1_score": f1_score})
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 batch_normalization (BatchN  (None, 1, 1, 512)        2048      
 ormalization)                                                   
                                                                 
 gaussian_noise (GaussianNoi  (None, 1, 1, 512)        0         
 se)                                                             
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                        

In [4]:
# define class
classes=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']

In [6]:
# loading caffe model for detecting face
modelFile ="models/res10_300x300_ssd_iter_140000.caffemodel"
configFile ="models/deploy.prototxt.txt"

net = cv2.dnn.readNetFromCaffe( configFile, modelFile)

In [ ]:
# prevents openCL usage and unnecessary logging messages
cv2.ocl.setUseOpenCL(False)

# dictionary which assigns each label an emotion (alphabetical order)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cam = cv2.VideoCapture(0)
time.sleep(2)


while True:
    ret, frame = cam.read()
    if not ret:
        break
    frame = imutils.resize(frame, width=700,height=700)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
    (h, w) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300),
                    mean=(104.0, 117.0, 123.0), swapRB=False, crop=False)
    net.setInput(blob)
    detections = net.forward()
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            y = startY - 10 if startX - 10 > 10 else startY+10
            
            cv2.rectangle(frame, (startX,startY), (endX,endY),(0,0,255),2)
            
            face = gray[startY:endY, startX:endX]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(face, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


# do a bit of cleanup
cv2.destroyAllWindows()

1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 69ms/step
